In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

In [2]:
# device = torch.device("cpu")
device = torch.device("cuda:0")

In [3]:
def get_char_sequence(word,char_to_idx):
    return [char_to_idx[i] for i in word]

def generate_sequence(seq,word_to_idx,char_to_idx):
#     print(word_to_idx[seq[0]],get_char_sequence(seq[0],char_to_idx))
    return [word_to_idx[i] for i in seq]

# def prepare_target(tags, pos_map):
#     return autograd.Variable(torch.LongTensor([pos_map[i] for i in tags]))

def prepare_target(tag, words_to_idx):
    return autograd.Variable(word_to_idx[tag])

In [3]:
# training_data = [
#     ["The dog ate the apple".lower().split(), ["DET", "NN", "V", "DET", "NN"]],
#     ["Everybody read that book".lower().split(), ["NN", "V", "DET", "NN"]],
#     ["a clever brown fox jumps over the lazy dog".lower().split(),["DET","JJ","JJ","NN","V","IN","DET","JJ","NN"]]
# ]
from pickle import dump
from pickle import load
import pickle
with open("telugu_pos_tag_data.pickle","rb") as fhnd:
    total_data=pickle.load(fhnd)

# Shuffling section

In [4]:
# from random import shuffle

# shuffle(total_data)
from pickle import dump
from pickle import load

# Creating word , char dictionaries , Don't run , they are created already


In [7]:
word_to_idx,char_to_idx={},{}

pos_map={}

sentences=[]
labels=[]

target_task_indx_start=0
target_task_indx_end=0

length = 5


for m,(sent,tag) in enumerate(total_data):
    sentences+=sent
    labels+=tag
    
    if m==5000:
        print("Starting Index of target task(1000 sentences) is ",(len(sentences)-length))
        target_task_indx_start=len(sentences)-length
    if m==6000:
        print("Ending Index of target task(1000 sentences) is ",len(sentences)-length)
        target_task_indx_end=len(sentences)-length
    
    for j in sent:
        if j not in word_to_idx.keys():
            word_to_idx[j] = len(word_to_idx)

        for k in j:
            if k not in char_to_idx.keys():
                char_to_idx[k] = len(char_to_idx)
    
    for tg in tag:
        if tg not in pos_map.keys():
            pos_map[tg] = len(pos_map)


Starting Index of target task(1000 sentences) is  64078
Ending Index of target task(1000 sentences) is  77677


In [4]:
dump(word_to_idx,open("word_to_idx.pickle","wb"))
dump(char_to_idx,open("char_to_idx.pickle","wb"))
dump(pos_map,open("pos_map.pickle","wb"))


# Run this cell to load word_to_idx , char_to_idx , pos_map

In [5]:
word_to_idx=load(open("word_to_idx.pickle","rb"))
char_to_idx=load(open("char_to_idx.pickle","rb"))

pos_map=load(open("pos_map.pickle","rb"))

# Don't run the below cells

In [35]:


sequences = list()
labelsequences=list()




for i in range(length, len(sentences)):
    # select sequence of tokens
    seq = sentences[i-length:i+1]
    tagseq=labels[i-length:i+1]
    

    # store
    sequences.append(seq)
    labelsequences.append(tagseq)
    
print('Total Sequences: %d' % len(sequences))


Total Sequences: 103590


In [36]:
print(sequences[:2])

[['గ్రేటర్', 'నోయిడా', 'పశ్చిమంలోని', 'మురుగునీటిని', 'తగ్గించే', 'ప్రస్తావన'], ['నోయిడా', 'పశ్చిమంలోని', 'మురుగునీటిని', 'తగ్గించే', 'ప్రస్తావన', 'చేయడం']]


In [37]:
print(labelsequences[:2])

[['N-NNP', 'N-NNP', 'N_NN', 'N_NN', 'V_VM_VNF', 'N_NN'], ['N-NNP', 'N_NN', 'N_NN', 'V_VM_VNF', 'N_NN', 'V_VM_VNG']]


In [38]:
print(total_data[:2])

[(['గ్రేటర్', 'నోయిడా', 'పశ్చిమంలోని', 'మురుగునీటిని', 'తగ్గించే', 'ప్రస్తావన', 'చేయడం', 'లేదు', '.'], ['N-NNP', 'N-NNP', 'N_NN', 'N_NN', 'V_VM_VNF', 'N_NN', 'V_VM_VNG', 'V_VM_VF', 'RD_PUNC']), (['దీని', 'అధీనంలో', 'మిత్రకీటకాల', 'ద్వారా', 'తెల్లపురుగుల', 'అంతం', 'చేయబడుతుంది', '.'], ['PR_PRP', 'N_NN', 'N_NN', 'PSP', 'N_NN', 'N_NN', 'V_VM_VF', 'RD_PUNC'])]


In [39]:
from pickle import dump

out_filename = 'tel_sent_sequences.pickle'

dump(sequences,open(out_filename,"wb"))


In [40]:
out_filename = 'tel_tag_sequences.pickle'

dump(labelsequences,open(out_filename,"wb"))


## Run this to get sentences and labelsentences like "w1,w2,w3,w4,w5" and "tag_1,tag_2,tag_3,tag_4,tag_5"

In [41]:
from pickle import load
in_filename = 'tel_sent_sequences.pickle'
lines= load(open(in_filename,'rb'))


In [42]:
from pickle import load
in_filename = 'tel_tag_sequences.pickle'
labellines= load(open(in_filename,'rb'))


# Don't run the below 4 cells . They convert "w1,w2,w3,w4,w5" to "w1_id,w2_id,w3_id,w4_id,w5_id"

In [43]:
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [word_to_idx[wrd] for wrd in line]
	# store
	sequences.append(encoded_seq)

In [44]:
from pickle import dump
out_filename = 'sequences.pickle'
dump(sequences,open(out_filename,'wb'))

In [45]:
labelsequences = list()
for line in labellines:
	# integer encode line
	encoded_seq = [pos_map[wrd] for wrd in line]
	# store
	labelsequences.append(encoded_seq)

In [47]:
from pickle import dump
out_filename = 'labelsequences.pickle'
dump(sequences,open(out_filename,'wb'))

# Run the below cell to get vocab_size

In [6]:

vocab_size = len(word_to_idx)
print('Vocabulary Size: %d' % vocab_size)


Vocabulary Size: 23138


In [48]:
print(len(sequences))
print(len(labelsequences))

103590
103590


In [49]:
sequences[:4]

[[0, 1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 6],
 [2, 3, 4, 5, 6, 7],
 [3, 4, 5, 6, 7, 8]]

# Don't run the below 3 cells . These cells convert "w1_id,w2_id,w3_id,w4_id,w5_id" to "w1_id,w2_id,w3_id,w4_id"  in phase1_X and "w5" in phase1_y 

# And phase2_X contains "w1_id,w2_id,w3_id,w4_id" and phase2_y contains "tag_1_id,tag_2_id,tag_3_id,tag_4_id"

In [50]:
import numpy as np
from numpy import array
sequences = array(sequences)

phase1_X=[]
phase1_y=[]

phase2_X=[]
phase2_y=[]



for sqnc,labeells in zip(sequences,labelsequences):
    phase1_X.append(sqnc[:-1])
    phase1_y.append(sqnc[-1])
#     phase2_y.append(str(sqnc[-1])+"_"+str(labeells[-1]))
    phase2_X.append(torch.tensor(sqnc[:-1],dtype=torch.long))
    phase2_y.append(torch.tensor(labeells[:-1],dtype=torch.long))

phase1_X=array(phase1_X)
phase1_y=array(phase1_y)

phase2_X=phase2_X
phase2_y=phase2_y

In [51]:
print(phase2_X[:2],phase2_y[:2])
print(phase1_X[:2],phase1_y[:2])

print(len(phase2_X))

[tensor([ 0,  1,  2,  3,  4]), tensor([ 1,  2,  3,  4,  5])] [tensor([ 0,  0,  1,  1,  2]), tensor([ 0,  1,  1,  2,  1])]
[[0 1 2 3 4]
 [1 2 3 4 5]] [5 6]
103590


In [52]:
from pickle import dump


with open("phase1_X.pickle","wb") as fhand:
    dump(phase1_X,fhand)

with open("phase2_X.pickle","wb") as fhand:
    dump(phase2_X,fhand)

with open("phase1_y.pickle","wb") as fhand:
    dump(phase1_y,fhand)

with open("phase2_y.pickle","wb") as fhand:
    dump(phase2_y,fhand)


# Run the below cell to load phase1_X,phase2_X,phase1_y,phase2_y

In [7]:
with open("phase1_X.pickle","rb") as fhand:
    phase1_X=load(fhand)

with open("phase2_X.pickle","rb") as fhand:
    phase2_X=load(fhand)

with open("phase1_y.pickle","rb") as fhand:
    phase1_y=load(fhand)

with open("phase2_y.pickle","rb") as fhand:
    phase2_y=load(fhand)


# Don't run the below 4 cells b'coz they are stored already. "phase1_encoded_total_data" contains [[w1_id,w2_id,w3_id,w4_id],w5_id] 

# "phase2_encoded_total_data" contains [[[w1_id,w2_id,w3_id,w4_id],[tag1_id,tag2_id,tag3_id,tag4_id]]]

In [53]:
phase1_encoded_total_data=[]
for text,targ in zip(phase1_X,phase1_y):
    phase1_encoded_total_data.append((text,targ))


In [54]:
phase2_encoded_total_data=[]
for text,targ in zip(phase2_X,phase2_y):
    phase2_encoded_total_data.append((text,targ))


In [57]:
with open("phase2_encoded_total_data.pickle","wb") as fhand:
    dump(phase2_encoded_total_data,fhand)

In [58]:
with open("phase1_encoded_total_data.pickle","wb") as fhand:
    dump(phase1_encoded_total_data,fhand)
    

# Run the below to load phase1,2_encoded_total_data

In [7]:
from pickle import load
with open("phase2_encoded_total_data.pickle","rb") as fhand:
    phase2_encoded_total_data=load(fhand)

In [8]:
with open("phase1_encoded_total_data.pickle","rb") as fhand:
    phase1_encoded_total_data=load(fhand)

## Run the cells below ,to_categorical() function converts a label to one-hot vector representation.  

In [8]:
import numpy as np
def to_categorical(number,num_classes):
    arr=torch.zeros((1,num_classes),dtype=torch.float)
#     print(arr)
    arr[0][number]=1
    return arr

# Converting the data to tensors . 

# [(tensor([ 0,  1,  2,  3,  4]), tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])) is one entry of phase1_encoded_catg_total_data

In [10]:
phase1_encoded_catg_total_data=[]
for i,(sns,targets) in enumerate(phase1_encoded_total_data):
    phase1_encoded_catg_total_data .append((torch.tensor(sns,dtype=torch.long),to_categorical(targets, num_classes=vocab_size)))

In [11]:
del phase1_encoded_total_data

In [ ]:
print(phase1_encoded_catg_total_data[:2])
print(phase1_encoded_total_data[:2])
print(len(phase1_encoded_catg_total_data))

print(phase1_encoded_catg_total_data[0][1].shape)


# Don't run the below 5 cells

In [14]:
phase1_catg_enc_training_data=phase1_encoded_catg_total_data[:len(phase1_encoded_catg_total_data)-1522]
phase1_catg_enc_test_data=phase1_encoded_catg_total_data[-1522:]

In [70]:
phase2_catg_enc_training_data=phase2_encoded_total_data[:len(phase2_encoded_total_data)-1522]
phase2_catg_enc_test_data=phase2_encoded_total_data[-1522:]

In [65]:
import torch.utils.data

In [67]:
phase1_trainloader=torch.utils.data.DataLoader(phase1_catg_enc_training_data, batch_size=64, shuffle=False)
phase1_testloader=torch.utils.data.DataLoader(phase1_catg_enc_test_data, batch_size=64, shuffle=False)

In [69]:
# phase2_trainloader=torch.utils.data.DataLoader(phase2_catg_enc_training_data, batch_size=32, shuffle=False, num_workers=8)
phase2_trainloader=torch.utils.data.DataLoader(phase2_catg_enc_training_data, batch_size=64, shuffle=False)
phase2_testloader=torch.utils.data.DataLoader(phase2_catg_enc_test_data, batch_size=64, shuffle=False)

# Don't run the below 4 cells as it is creating memory error

In [ ]:
from pickle import dump
out_filename = 'phase1_catg_enc_training_data.pickle'
dump(phase1_catg_enc_training_data,open(out_filename,'wb'))



In [ ]:
from pickle import dump
out_filename = 'phase1_catg_enc_test_data.pickle'
dump(phase1_catg_enc_test_data,open(out_filename,'wb'))


In [ ]:

from pickle import dump
out_filename = 'phase2_catg_enc_test_data.pickle'
dump(phase2_catg_enc_test_data,open(out_filename,'wb'))



In [ ]:
from pickle import dump
out_filename = 'phase2_catg_enc_training_data.pickle'
dump(phase2_catg_enc_training_data,open(out_filename,'wb'))

In [ ]:
# from pickle import load
# in_filename = 'phase1_catg_enc_training_data.pickle'
# phase1_catg_enc_training_data=load(open(in_filename,'rb'))

# in_filename = 'phase1_catg_enc_test_data.pickle'
# phase1_catg_enc_test_data=load(open(in_filename,'rb'))

# in_filename = 'phase2_catg_enc_test_data.pickle'
# phase2_catg_enc_test_data=load(open(in_filename,'rb'))

# in_filename = 'phase2_catg_enc_training_data.pickle'
# phase2_catg_enc_training_data=load(open(in_filename,'rb'))

# Run the below cells

In [7]:
word_embedding_dim = 200
# char_embedding_dim = 50
hidden_dim = 450

In [8]:
# pos_map={'DET':0,'NN':1,'V':2,'JJ':3,'IN':4}

phase=1

In [9]:
class pos_lstm_model(nn.Module):
    def __init__(self,hidden_dim,embedding_dim,vocab_size,pos_tag_size):
        super(pos_lstm_model,self).__init__()
#         self.nn = nn.cuda()
        
        self.embeddings=nn.Embedding(vocab_size,embedding_dim)   # Embedding layer with output dim as 200 and input is #words
        self.layer1 = nn.LSTM(embedding_dim,hidden_dim)          # LSTM with output dimension as 450 and input dim as 200 
        self.layer2 = nn.Linear(hidden_dim,vocab_size)  
        # Dense layer for predicting the next word ,input is last hidden state. Output is [1,23138]

        self.phase2_layer = nn.Linear(hidden_dim,pos_tag_size) 
        # Dense layer for predicting the pos tags ,input is hidden states of the time steps and output of [#words,#pos_tags]
        
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden() 
        
    def init_hidden(self):
        return (torch.zeros(1,1,hidden_dim,device=device),torch.zeros(1,1,hidden_dim,device=device))
        
    def forward(self,sentence):
        
        embed = self.embeddings(sentence) # Output Dim is [#words,#embedding_size]
        
        output1,self.hidden = self.layer1(embed.view(len(sentence),1,-1),self.hidden)
        
        # Input dim is [#words,1,#embedding_size] . Output is hidden states of each word. And self.hidden is last hidden state
        
        # Output dim is [#words,1,#hidden_dim] 
        
        output2=self.layer2(output1[-1])
        
        #Input the last hidden state into the dense layer to predict the next word
        
        if phase==2:
            phase2_output=self.phase2_layer(output1.view(len(sentence),-1))
            # Here, we input all hidden states i,e same as #words and get output as [#words,#pos_tags]
            
            return F.log_softmax(output2,dim=1),F.log_softmax(phase2_output,dim=1)  
            # Returning both predicted word and pos tags of the input words 
            
        
        return F.log_softmax(output2,dim=1)

In [29]:
model = pos_lstm_model(hidden_dim,word_embedding_dim,len(word_to_idx),len(pos_map))
model=model.cuda()
loss_function = nn.MultiLabelSoftMarginLoss().cuda() # For Language model 

# loss_function = nn.BCEWithLogitsLoss().cuda()

loss_function2= nn.NLLLoss().cuda()  # For POS tagging
OPTIMIZER = optim.SGD(model.parameters(),lr = 0.01)

In [13]:
no_epochs = 20

In [31]:
print(len(phase1_encoded_catg_total_data))

103590


In [32]:
for i in range(no_epochs):  
    if i%50 == 0:
        print("epoch ", i, ' of ', no_epochs)
        
    print("epoch ", i, ' of ', no_epochs)
        
    if i>0:
        print("task1_phase2_loss is ",task1_phase2_loss)
        print("task2_phase2_loss is ",task2_phase2_loss)
        
    count2=0
    for count,(sentence, tag) in enumerate(phase1_encoded_catg_total_data[:target_task_indx_end]):

        
        model.zero_grad()

        model.hidden = model.init_hidden()

        sentence_in=sentence.cuda() # Convert sentence tensor([w1_id,w2_id,w3_id,w4_id]) to GPU tensor
        
        targets=tag  # targets variable is [0,0,0,1,0,0,0,0,0,0....] for 5th indexed word in word_to_index dictionary
        
        
        
        if count>=target_task_indx_start:  # target_task_indx_start stores the index at which target data starts
            phase=2
            count2+=1
            phase2_sent,phase2_tag=phase2_encoded_total_data[count] 
            
            #phase2_sent is tensor([w1_id,w2_id,w3_id,w4_id])
            
            #phase2_tag is tensor([tag1_id,tag2_id,tag3_id,tag4_id])
            
            phase2_sent=phase2_sent.cuda()
            phase2_tag=phase2_tag.cuda()
            targets=targets.cuda()
            tag_scores1,tag_scores2 = model(phase2_sent) 
            # As told in model , we have tag_scores1 storing the next word with dimension as [1,vocab_size]
            
            # tag_scores_2 stores the pos tags of incoming words with dimension as [#words,#pos tags]
            
            loss = loss_function(tag_scores1,targets)+loss_function2(tag_scores2,phase2_tag)
            
            #Adding both the losses
            task1_phase2_loss=loss_function(tag_scores1,targets)
            task2_phase2_loss=loss_function2(tag_scores2,phase2_tag)
#             print("Phase 2 source task loss is",task1_phase2_loss,"Target task loss is",task2_phase2_loss)
            loss.backward()
            
            OPTIMIZER.step()
            
            
        
        else:
            phase=1
            tag_scores = model(sentence_in)
            targets=targets.cuda()


            phase1_loss = loss_function(tag_scores,targets)
#             if count==0:
#                 phase1_loss.backward(retain_graph=True)
#             else:
#             print("Phase 1 source task loss is ",phase1_loss)
            phase1_loss.backward()
            OPTIMIZER.step()
            
            
print("done")        

epoch  0  of  20
epoch  0  of  20
epoch  1  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.6926, device='cuda:0')
task2_phase2_loss is  tensor(0.9749, device='cuda:0')
epoch  2  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.6895, device='cuda:0')
task2_phase2_loss is  tensor(0.6364, device='cuda:0')
epoch  3  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.6713, device='cuda:0')
task2_phase2_loss is  tensor(0.2983, device='cuda:0')
epoch  4  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.6443, device='cuda:0')
task2_phase2_loss is  tensor(0.1748, device='cuda:0')
epoch  5  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.6159, device='cuda:0')
task2_phase2_loss is  tensor(0.1114, device='cuda:0')
epoch  6  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.5873, device='cuda:0')
task2_phase2_loss is  tensor(1.00000e-02 *
       7.6929, device='cuda:0')
epoch  7  of  20
task1_phase2_loss is  tensor(1.00000e-04 *
       4.

In [ ]:
# for i in range(no_epochs):  
#     if i%50 == 0:
#         print("epoch ", i, ' of ', no_epochs)
        
#     print("epoch ", i, ' of ', no_epochs)
        
#     if i>0:
#         print("Loss is ",loss)
    
#     count2=0
#     for count, data in enumerate(phase1_trainloader, 0):
        
#         sentence, tag = data    
# #         inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())        
        
#         model.zero_grad()

#         model.hidden = model.init_hidden()

#         sentence_in=sentence.cuda()
        
#         targets=tag
        
        
        
#         if count>=99999:
#             phase=2
#             count2+=1
#             phase2_sent,phase2_tag=phase2_catg_enc_training_data[count]
#             phase2_sent=phase2_sent.cuda()
#             phase2_tag=phase2_tag.cuda()
#             targets=targets.cuda()
#             tag_scores1,tag_scores2 = model(phase2_sent)
#             loss = loss_function(tag_scores1,targets)+loss_function2(tag_scores2,phase2_tag)
#             loss.backward()
                
#             OPTIMIZER.step()
            
            
        
#         else:
#             tag_scores = model(sentence_in)
#             targets=targets.cuda()
# #             print(targets)
#             print("Tag scores is ",tag_scores[0].shape)
#             print(tag_scores[0])
#             loss = loss_function(tag_scores[0],targets)
#             if count==0:
#                 loss.backward(retain_graph=True)
#             else:
#                 loss.backward()
#             OPTIMIZER.step()
            
            
# print("done")        

In [42]:
from pickle import dump

In [33]:

torch.save(model,'model_LM_plus_pos_char_{}epochs.pt'.format(no_epochs))


C:\Users\kmura\Anaconda3\lib\site-packages\torch\serialization.py:193: UserWarning: Couldn't retrieve source code for container of type pos_lstm_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [10]:
no_epochs=20
model = torch.load('model_LM_plus_pos_char_{}epochs.pt'.format(no_epochs))

In [ ]:
with torch.no_grad():
    
    sentence_in = generate_sequence(training_data[1000][0], word_to_idx,char_to_idx)
        
    targets = prepare_target(training_data[1000][1], pos_map)
    tag_scores = model(sentence_in)
    print(tag_scores.shape)
    print(targets.shape)

In [11]:
target_task_indx_end=77677
target_task_indx_start=64078

In [12]:
def get_sequence(seq,dictionary):
    return torch.tensor([dictionary[i] for i in seq], dtype = torch.long)

In [14]:
testing_data=total_data[-2022:]

In [18]:
total=0
correct=0
phase=2
for count,(sentence, tag) in enumerate(total_data[:5000]):
#     flatten_parameters()
    print("sentence ",count)
    model.zero_grad()

    model.hidden = model.init_hidden()
    sentence = get_sequence(sentence,word_to_idx)
    tag= get_sequence(tag,pos_map)
    sentence_in=sentence.cuda() # Convert sentence tensor([w1_id,w2_id,w3_id,w4_id]) to GPU tensor
    targets=tag  # targets variable is [0,0,0,1,0,0,0,0,0,0....] for 5th indexed word in word_to_index dictionary
        
        
    targets=targets.cuda()
    
    tag_scores1,tag_scores2 = model(sentence_in) 
            # As told in model , we have tag_scores1 storing the next word with dimension as [1,vocab_size]
            
            # tag_scores_2 stores the pos tags of incoming words with dimension as [#words,#pos tags]
            
    _, predicted = torch.max(tag_scores2.data, 1)
    print(targets)
    print(predicted)
    total += targets.size(0)

    correct += (predicted == targets).sum().item()
print('Accuracy of the network on the {} test images: %d %%'.format(count) % (100 * correct / total))


sentence  0


C:\Users\kmura\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


tensor([ 0,  0,  1,  1,  2,  1,  3,  4,  5], device='cuda:0')
tensor([ 1,  1,  1,  4,  1,  1,  3,  4,  5], device='cuda:0')
sentence  1
tensor([ 6,  1,  1,  7,  1,  1,  4,  5], device='cuda:0')
tensor([ 6,  1,  1,  7,  0,  1,  4,  5], device='cuda:0')
sentence  2
tensor([ 1,  1,  7,  1,  2,  5,  8,  2,  8,  7,  6,  1,  9,  8,
         8,  8,  8,  1,  1,  4,  5], device='cuda:0')
tensor([ 1,  1,  7,  1,  3,  5,  0,  2,  2,  7,  6,  1,  9,  8,
         8,  1,  1,  1,  4,  1,  5], device='cuda:0')
sentence  3
tensor([  1,   1,  10,   1,   1,   1,   1,   2,  11,  12,   1,   4,
          5], device='cuda:0')
tensor([  1,   1,  10,   1,   1,   4,   3,   2,   4,  12,   1,   1,
          5], device='cuda:0')
sentence  4
tensor([  0,  12,   1,   1,   1,   1,   1,  10,   1,   1,  12,   1,
          5], device='cuda:0')
tensor([  8,  12,   1,   1,   1,   4,   1,  10,   1,   1,   1,   1,
          5], device='cuda:0')
sentence  5
tensor([  6,   1,   8,   1,   1,   2,   1,   1,   1,  11,   1,   7,


tensor([ 10,  12,  10,   9,   1,   2,  10,   2,   1,   1,   3,   1,
          1,   4,   5], device='cuda:0')
tensor([ 10,  12,  10,   9,   1,   2,  10,   1,   1,   1,   3,   1,
          1,   4,   5], device='cuda:0')
sentence  60
tensor([ 10,  12,   1,   1,   1,   4,   5], device='cuda:0')
tensor([ 10,   1,   1,   1,   2,   1,   5], device='cuda:0')
sentence  61
tensor([  1,  12,   1,  14,   1,   2,   1,   1,  10,   9,   1,   4,
          5,   6,   1,   9,   1,   4,   5], device='cuda:0')
tensor([  1,   1,   1,  14,  12,   4,   1,   1,  10,   9,   1,   4,
          5,   6,   1,   9,   1,   4,   5], device='cuda:0')
sentence  62
tensor([ 10,   1,   9,   4,   5], device='cuda:0')
tensor([ 10,   1,   9,   1,   5], device='cuda:0')
sentence  63
tensor([ 10,   6,   1,   1,  20,   4,   5], device='cuda:0')
tensor([ 10,   1,   1,   1,  20,   1,   5], device='cuda:0')
sentence  64
tensor([  1,  11,   3,   1,   1,   1,   0,   1,   4,   5,   6,   0,
          1,   4,   5], device='cuda:0')
tens

tensor([ 12,   1,  15,   9,   1,   1,   5,   1,   4,   5], device='cuda:0')
tensor([ 12,   1,  15,   9,   1,   4,   5,   1,   4,   5], device='cuda:0')
sentence  120
tensor([ 6,  7,  9,  1,  9,  8,  8,  1,  4,  5], device='cuda:0')
tensor([ 6,  7,  9,  1,  9,  8,  8,  1,  1,  5], device='cuda:0')
sentence  121
tensor([  1,   1,   5,   1,   2,   1,   1,   1,  16,   1,   1,   4,
          5], device='cuda:0')
tensor([  1,   1,   5,   1,   1,   1,   1,   1,  11,   1,   1,   1,
          5], device='cuda:0')
sentence  122
tensor([  1,   1,   7,   9,   1,  10,   1,   3,  16,   1,   3,  16,
         12,  11,   4,   5], device='cuda:0')
tensor([  1,   1,   7,   9,   1,  10,   1,   1,  11,   1,   1,   4,
          1,   1,   1,   5], device='cuda:0')
sentence  123
tensor([ 10,   1,   1,   1,   1,   1,   2,   9,   1,   4,   5], device='cuda:0')
tensor([ 10,   1,   3,   1,   1,   1,   2,   9,   1,   1,   5], device='cuda:0')
sentence  124
tensor([  0,  10,   0,   1,   1,   2,  11,   2,   1,   5, 

...

tensor([ 15,   2,   1,   1,  12,   1,   3,   8,   1,   1,   3,   1,
          5], device='cuda:0')
tensor([ 15,   2,   1,   1,  12,   1,   3,   1,   4,   1,   3,   1,
          5], device='cuda:0')
sentence  4719
tensor([ 0,  0,  1,  3,  8,  0,  1,  0,  1,  4,  5], device='cuda:0')
tensor([  1,   1,   1,   3,  15,   0,   1,   1,  15,   4,   5], device='cuda:0')
sentence  4720
tensor([ 14,   1,  10,   1,   7,   1,   4,  11,   6,   7,  12,   1,
          1,   8,   1,   1,   4,   5], device='cuda:0')
tensor([ 14,   1,  10,   1,   7,   1,   4,   4,   6,   7,  12,   1,
          1,   8,   1,   1,   4,   5], device='cuda:0')
sentence  4721
tensor([  1,   1,   1,   1,   0,  25,  15,   1,   1,   1,   1,   0,
          0,   1,   1,   9,   3,   1,   1,   1,   4,   5], device='cuda:0')
tensor([  1,   1,   1,   1,   1,   0,  15,   1,   1,   1,   0,   1,
         11,   1,   1,   9,   0,   1,   4,   1,   4,   5], device='cuda:0')
sentence  4722
tensor([  1,   1,   1,   3,   8,  12,   1,   4,   5,  1

tensor([  2,   1,  10,   1,   7,   1,   8,   1,   2,   1,  10,   6,
         14,   1,   9,   1,   1,   4,   5,  10,   1,   6,   1,   1,
         10,   8,   1,   1,   1,   4,   5], device='cuda:0')
tensor([ 12,   1,  10,   1,   7,   1,   8,   1,   2,   1,  10,   6,
         14,   1,   9,   1,   1,   4,   5,  10,   8,   6,   2,   1,
         10,  15,   1,   1,   1,   4,   5], device='cuda:0')
sentence  4770
tensor([ 6,  1,  1,  1,  2,  1,  5], device='cuda:0')
tensor([ 6,  1,  4,  4,  1,  1,  5], device='cuda:0')
sentence  4771
tensor([  6,   1,   2,   1,   4,   5,  10,   6,  11,   2,  11,   2,
          4,   5], device='cuda:0')
tensor([  6,   1,   4,   1,   1,   5,  10,   6,  18,   1,  11,   4,
          4,   5], device='cuda:0')
sentence  4772
tensor([ 12,   1,   6,   4,   5], device='cuda:0')
tensor([ 3,  1,  6,  1,  5], device='cuda:0')
sentence  4773
tensor([  1,   1,  15,   3,  16,   4,   5], device='cuda:0')
tensor([  1,   1,  15,  15,  16,   1,   5], device='cuda:0')
sentence  4

tensor([ 1,  7,  1,  1,  1,  2,  1,  1,  5], device='cuda:0')
sentence  4833
tensor([ 1,  8,  7,  8,  7,  1,  1,  4,  5], device='cuda:0')
tensor([ 1,  8,  7,  1,  7,  1,  1,  4,  5], device='cuda:0')
sentence  4834
tensor([ 14,   1,   2,   1,   1,   4,   5], device='cuda:0')
tensor([ 14,   1,   2,   1,   3,   4,   5], device='cuda:0')
sentence  4835
tensor([  0,   0,   1,  11,   4,   5], device='cuda:0')
tensor([ 1,  3,  1,  1,  4,  5], device='cuda:0')
sentence  4836
tensor([ 14,   1,   0,   5,   0,   0,   5,   0,   5,   0,   5,   0,
         10,  12,   1,   1,   7,   1,   4,   5], device='cuda:0')
tensor([ 14,   1,   1,   5,   1,   4,   5,   1,   5,   1,   5,   1,
         10,   1,   1,   4,   7,   1,   4,   5], device='cuda:0')
sentence  4837
tensor([  6,   1,   1,  16,   4,   5], device='cuda:0')
tensor([  6,   1,   1,  16,   4,   5], device='cuda:0')
sentence  4838
tensor([  0,   1,   1,   1,   2,   1,  11,   2,   4,   5], device='cuda:0')
tensor([  4,   1,  11,   1,   2,   1,  1

sentence  4901
tensor([ 1,  1,  9,  1,  1,  1,  4,  5], device='cuda:0')
tensor([ 1,  1,  9,  1,  1,  1,  1,  5], device='cuda:0')
sentence  4902
tensor([  1,   1,   0,   1,   2,   1,   0,   5,   0,   6,  12,   1,
          4,   4,   5], device='cuda:0')
tensor([ 1,  1,  1,  1,  1,  1,  0,  5,  1,  6,  9,  1,  2,  1,
         5], device='cuda:0')
sentence  4903
tensor([  0,   1,  12,   1,   1,   1,   1,   1,   2,   4,   5,  10,
          6,   1,  11,   4,  10,   0,   1], device='cuda:0')
tensor([  0,   0,   9,   1,   9,   2,   4,   1,   2,   4,   5,  10,
          6,   1,  11,   4,  10,  11,   1], device='cuda:0')
sentence  4904
tensor([  1,   1,   1,   1,   4,   5,   6,   1,   1,  15,   2,   4,
          5], device='cuda:0')
tensor([ 1,  1,  1,  1,  1,  5,  6,  4,  1,  7,  2,  4,  5], device='cuda:0')
sentence  4905
tensor([  9,   1,   6,   1,  16,   4,   5], device='cuda:0')
tensor([  9,   1,   6,  15,  16,   4,   5], device='cuda:0')
sentence  4906
tensor([  3,  15,   1,   4,   5,  

tensor([  1,  11,   1,   3,  10,  11,   1,   3,   1,  12,  12,   1,
          4,   5], device='cuda:0')
tensor([  1,  11,   1,   3,  10,   1,   1,   1,   1,  12,  12,   1,
          4,   5], device='cuda:0')
sentence  4965
tensor([  1,   1,   7,   1,  10,   6,   1,   1,   1,   5,   1,   2,
          1,   5,   1,   1,  10,   1,  10,   1,   1,   7,   4,   5], device='cuda:0')
tensor([  1,   1,   7,   1,  10,   1,   8,   1,   1,   5,   1,   1,
          1,   5,   1,   6,  10,   1,  10,   1,   1,   7,   1,   5], device='cuda:0')
sentence  4966
tensor([ 0,  8,  1,  1,  4,  5], device='cuda:0')
tensor([ 1,  8,  1,  1,  1,  5], device='cuda:0')
sentence  4967
tensor([ 6,  1,  1,  4,  5], device='cuda:0')
tensor([ 6,  6,  2,  4,  5], device='cuda:0')
sentence  4968
tensor([  1,   1,   1,  11,   4,  11,   6,   1,   7,  12,   1,   4,
          5], device='cuda:0')
tensor([  4,   0,   4,   1,   4,  11,   6,   1,   7,   1,   1,   4,
          5], device='cuda:0')
sentence  4969
tensor([ 14,   1,  

In [ ]:
total=0
correct=0
phase=2
for count,(sentence, tag) in enumerate(phase2_encoded_total_data[target_task_indx_end:]):
    
    print("sentence ",count)
        
    model.zero_grad()

    model.hidden = model.init_hidden()

    sentence_in=sentence.cuda() # Convert sentence tensor([w1_id,w2_id,w3_id,w4_id]) to GPU tensor
        
    targets=tag  # targets variable is [0,0,0,1,0,0,0,0,0,0....] for 5th indexed word in word_to_index dictionary
        
        
    targets=targets.cuda()
    
    tag_scores1,tag_scores2 = model(sentence_in) 
            # As told in model , we have tag_scores1 storing the next word with dimension as [1,vocab_size]
            
            # tag_scores_2 stores the pos tags of incoming words with dimension as [#words,#pos tags]
            
    _, predicted = torch.max(tag_scores2.data, 1)
    print(targets)
    print(predicted)
    total += targets.size(0)

    correct += (predicted == targets).sum().item()
print('Accuracy of the network on the {} test images: %d %%'.format(count) % (100 * correct / total))


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for k in range(len(phase1_catg_enc_test_data)):
        
        tag_scores = model(sentence_in)
        print(tag_scores.shape)
#         print(targets.shape)
        _, predicted = torch.max(tag_scores.data, 1)
        
        print("Predicted is ",predicted)
        arrtargets=targets.data
        print("Targets is ,",arrtargets)
        _,targlabel=arrtargets.max(0)
        total += 1
        if targlabel==predicted:
            correct+=1
            

print('Accuracy of the network on the {} test images: %d %%'.format(len(test_data)) % (100 * correct / total))

        
# with torch.no_grad():
#     for data in test_data:
#         test_sent, test_tag = data
#         sentence_in = generate_sequence(test_sent, word_to_idx,char_to_idx)
#         targets = prepare_target(test_tag, pos_map)
#         outputs = model(sentence_in)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))